### The code and data are adapted from:  https://medium.com/@vitalshchutski/french-nlp-entamez-le-camembert-avec-les-librairies-fast-bert-et-transformers-14e65f84c148

In [ ]:
# !conda install torch
# !pip install fast-bert==1.9.1
# !mkdir model
# !mkdir finetuned_model

In [10]:
#!pip install nltk

  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434673 sha256=574c36068bf40223f1aacd1d1ce10988f0ca33bc84c9e79d97548443f120b7d7
  Stored in directory: c:\users\jiang\appdata\local\pip\cache\wheels\45\6c\46\a1865e7ba706b3817f5d1b2ff7ce8996aabdd0d03d47ba0266
Successfully built nltk


In [1]:
import torch
from fast_bert.data_cls import BertDataBunch 
from fast_bert.learner_cls import BertLearner
from fast_bert.data_lm import BertLMDataBunch
from fast_bert.learner_lm import BertLMLearner
from fast_bert.metrics import fbeta, roc_auc
from fast_bert.prediction import BertClassificationPredictor
from pathlib import Path
import pandas as pd
import logging


#logger = logging.getLogger()
device_cuda = torch.device("cuda")

In [2]:
logging.basicConfig(
    handlers=[logging.FileHandler('logfile.log', 'w', 'utf-8')],
    format='%(levelname)s: %(message)s',
    datefmt='%m-%d %H:%M',
    level=logging.INFO #CRITICAL ERROR WARNING  INFO    DEBUG    NOTSET 
)
logger = logging.getLogger()


In [3]:
DATA_PATH = Path('./data/')
LOG_PATH = Path('./logs/')
MODEL_PATH = Path('./model/')
LABEL_PATH = Path('./labels/')

In [4]:
df = pd.read_csv('./data/AA_xml_manul - 0-200_0-403.csv')

In [5]:
val_set = df.sample(frac=0.2, replace=False, random_state=42)
train_set = df.drop(index = val_set.index)
print('Nombre de commentaires dans le val_set:',len(val_set))
print('Nombre de commentaires dans le train_set:', len(train_set))
val_set.to_csv('./data/val_set.csv')
train_set.to_csv('./data/train_set.csv')

Nombre de commentaires dans le val_set: 80
Nombre de commentaires dans le train_set: 322


In [6]:
labels = df.columns[2:4].to_list() 
with open('./labels/labels.txt', 'w') as f:
    for i in labels:
        f.write(i + "\n")

In [4]:
#df_texts = pd.read_csv('./data/raw_xml_bsv_0-200.csv')
df_texts = pd.read_csv('./data/bsv_chunk256_raw_1001-1200.csv')

### text cleaning

In [5]:
import nltk
import re

# make all elements string
df_texts['report_text'] = df_texts['report_text'].astype(str)
# Remove null fields
df_texts['report_text'] = df_texts['report_text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# Make all text lowercase
df_texts['report_text'] = df_texts['report_text'].apply(lambda x: x.lower())
# Delete stop-words => to be tesred later
#stopwords = nltk.corpus.stopwords.words('french')


In [6]:
all_texts = df_texts['report_text'].to_list()[500:]
print('Nombre de bloc de texte:', len(all_texts))

Nombre de bloc de texte: 2301


### Création de LMDataBunch

In [7]:
all_texts[:10]

["orleans bulletin produit partir d’observations ponctuelles. donne tendance situation sanitaire régionale, peut être transposée telle quelle parcelle. chambre régionale d’agriculture centre dégage donc toute responsabilité quant décisions prises agriculteurs pour protection leurs cultures. %10%20%30%40%50%60%70%80%90%100% totalgénéralnb parcellesdépartementblé tendre d'hiver région centre3 nœuds2 nœuds1 nœudepi %10%20%30%40%50%60%70%80%90%100% totalgénéralnb parcellesdépartementblé tendre d'hiver région centre3 nœuds2 nœuds1 nœudepi malgré progression symptômes depuis semaine dernière, piétin verse toujours",
 'discret l’heure actuelle. observations faites cette semaine situations sont indemnes piétin verse. parcelle dessus seuil nuisibilité certaine tiges atteintes. situations plus touchées concernent toujours nord-est cher (cf. annexe). dans parcelles risque (retour fréquent blé, variété sensible, milieu favorable semis précoces). partir stade jusqu’à nœuds, déterminer pourcentage t

In [9]:
databunch_lm = BertLMDataBunch.from_raw_corpus(
                    data_dir=DATA_PATH,
                    text_list=all_texts,
                    tokenizer='camembert-base',
                    batch_size_per_gpu=8, #was 16, even 8 won't do
                    max_seq_length=256, #was 512
                    multi_gpu=False,
                    model_type='camembert-base',
                    logger=logger)

### Création de LMLearner

In [10]:
#torch.cuda.memory_stats()

In [ ]:
#torch.cuda.memory_summary()

In [10]:
lm_learner = BertLMLearner.from_pretrained_model(
                            dataBunch=databunch_lm,
                            pretrained_path='camembert-base',
                            output_dir=MODEL_PATH,
                            metrics=[],
                            device=device_cuda,
                            logger=logger,
                            multi_gpu=False,
                            logging_steps=50,
                            is_fp16=True) #was true with gpu

In [11]:
lm_learner.fit(epochs=1, #was 30
            lr=1e-4,
            validate=True,
            schedule_type="warmup_cosine",
            optimizer_type="adamw")

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0


C:\Users\Raja\Documents\SJI\tools\anaconda3\envs\fast-camembert\lib\site-packages\torch\optim\lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


C:\Users\Raja\Documents\SJI\tools\anaconda3\envs\fast-camembert\lib\site-packages\torch\optim\lr_scheduler.py:247: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


(36, 2.5813459985786014)

In [12]:
lm_learner.validate()

{'loss': 0.15649139881134033, 'perplexity': 1.169400691986084}

In [13]:
lm_learner.save_model()

In [14]:
del lm_learner

### Création de databunch pour la classification

In [15]:
databunch = BertDataBunch(DATA_PATH, LABEL_PATH,
                          tokenizer='camembert-base',
                          train_file='train_set.csv',
                          val_file='val_set.csv',
                          label_file='labels.txt',
                          text_col='report_text',
                          label_col=['Bioagressor','Disease'],
                          batch_size_per_gpu=8,
                          max_seq_length=256,
                          multi_gpu=False,
                          multi_label=True,
                          model_type='camembert-base')

### Création de Learner

In [16]:
metrics = [{'name': 'fbeta', 'function': fbeta}, {'name': 'roc_auc', 'function': roc_auc}]
OUTPUT_DIR = Path('./finetuned_model')
WGTS_PATH = Path('model/model_out/pytorch_model.bin')

In [17]:
# issue fast-bert pos_weight <= downgrade to 1.9.1 solve the prob
cl_learner = BertLearner.from_pretrained_model(
                        databunch,
                        pretrained_path='model/model_out',
                        metrics=metrics,
                        device=device_cuda, #was device_cuda
                        logger=logger,
                        output_dir=OUTPUT_DIR,
                        finetuned_wgts_path=WGTS_PATH,
                        warmup_steps=300,
                        multi_gpu=False,
                        multi_label=True,
                        is_fp16=True,#True when is cuda
                        logging_steps=50)

In [18]:
cl_learner.fit(epochs=10,# was 30
            lr=2e-5,
            validate=True,
            schedule_type="warmup_cosine",
            optimizer_type="adamw")

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 4096.0


(4310, 0.193165206902265)

In [19]:
cl_learner.validate()

{'loss': 0.5417303514701349,
 'fbeta': 0.38307493925094604,
 'roc_auc': 0.9060379297956141}

In [20]:
cl_learner.save_model()

In [21]:
del cl_learner

### Prédictions

In [22]:
predictor = BertClassificationPredictor(
                model_path='finetuned_model/model_out',
                label_path='labels/',
                multi_label=True,
                model_type='camembert-base',
                do_lower_case=False)

In [23]:
#cas disease: 0, bioagressor: 1 - cicadelle
predictor.predict("Cicadelles La cicadelle Edwardsiana est toujours observée sur les parcelles en été")

C:\Users\Raja\Documents\SJI\tools\anaconda3\envs\fast-camembert\lib\site-packages\pandas\core\frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


[('bioagressor', 0.98876953125), ('disease', 0.007965087890625)]

In [24]:
predictor.predict("election américane")

[('bioagressor', 0.00669097900390625), ('disease', 0.0037364959716796875)]

In [25]:
#cas disease: 1, bioagressor: 0 - mildiou
predictor.predict("vigilant : en particulier vis-à-vis du mildiou, de l’oïdium et de la bactériose / cladosporiose. Les prévisions pour les prochains jours restent peu favorables à l’expression de la bactériose et la cladosporiose, si elles se confirment. Mildiou (Pseudoperonospora cubensis) : Le modèle annonce un risque élevé pour toutes les dates de plantation avec les données de la station de Thurageau. Avec les données de la station de Maulay, les plantations en S25 et S26 montrent un risque modéré. Le risque est un peu plus élevé dans le sud de la Charente- Maritime que dans le Poitou. Niveau de risque Faible Moyen Élevé Très élevé Indice : Log (Nb de taches/unité de surface) -14 à -9 -9 à -4 -4 +4 Équivalent en unité de surface 1 tâche par hectare par 100 m2 1 tâche par 100 m2 par m2 1 tâche par m2à 1 % de surface atteinte 1 % à 100 % de surface atteinte Évaluation du risque : les conditions restent favorables à ce microorganisme (qui n’est pas un champignon, mais proche d’une algue). Les BSV sont disponibles en accès direct sur le site  (rubrique : Nos publications - Bulletin de santé du végétal) ou par abonnement en ligne gratuit sur le site  BSV CULTURES LÉGUMIÈRES DE PLEIN")

[('disease', 0.9912109375), ('bioagressor', 0.0199127197265625)]

In [26]:
#cas disease: 0, bioagressor: 0 - texte sur trump trump
predictor.predict("L'avance de Donald Trump dans cet Etat où 4,9 millions d'électeurs ont voté, a fondu vendredi 6 novembre. Le candidat républicain compte")

[('bioagressor', 0.00634002685546875), ('disease', 0.004039764404296875)]

In [27]:
#cas disease: 0, bioagressor: 0 - texte sur mouche de carotte mais pas de rique => no good
predictor.predict("mouche de la carotte :ajouter trichloronate")

[('disease', 0.00818634033203125), ('bioagressor', 0.0038394927978515625)]

In [28]:
#cas disease: 0, bioagressor: 0 - oidium : pas d'intervention => pas bon
predictor.predict("oïdium : pas d'intervention dans l'immediat les conditions très chaudes et l'absence de rosées nocturnes sont défavorables à cette maladie. aucun symptôme n'a encore été observé.")

[('disease', 0.98779296875), ('bioagressor', 0.0141754150390625)]

In [29]:
#cas disease: 1, bioagressor: 0 - oidium  => good
predictor.predict("oïdium du pommierce champignon est à l'heure actuelle en pleine fructification. il est nécessaire d'ajouter un antioïdium aux bouillies pour assurer'la protection du jeune feuillage.")

[('disease', 0.98974609375), ('bioagressor', 0.0173797607421875)]

In [30]:
#cas disease: 0, bioagressor: 0 - pucerons- le seuil de nuisibilité  n’est pas atteint => raté
predictor.predict("pucerons les pucerons verts sont toujours présents dans certaines parcelles. on note également la présence de sitobion avenae, relativement moins fréquente, mais avec des colonies de plusieurs dizaines d'individus. le temps frais actuel n'est pas favorable aux prédateurs, mais on observe un nombre important de pucerons «momifiés» par la ponte de parasitoïdes. le seuil de nuisibilité n'est atteint dans aucune parcelle du réseau")

[('bioagressor', 0.9912109375), ('disease', 0.010009765625)]

In [31]:
#cas disease: 0, bioagressor: 0 - zero mot clé
predictor.predict("par jour. Sur les autres secteurs, en production de carotte, les captures sont nulles. Évaluation du risque")

[('bioagressor', 0.006389617919921875), ('disease', 0.0038394927978515625)]

In [32]:
#cas disease: 0, bioagressor: 0 news + mildiou => raté
predictor.predict("7 Action pilotée par le ministère chargé de l’agriculture mildiou, avec l’appui financier de l’Office National de l’Eau et des Milieux Aquatiques (ONEMA), par les crédits issus de la redevance pour pollutions diffuses attribués au")

[('disease', 0.115966796875), ('bioagressor', 0.0016107559204101562)]

In [33]:
#cas disease: 0, bioagressor: 0
predictor.predict("financement du plan Ecophyto. Ce bulletin est rédigé par l'ACPEL avec la collaboration de référents par culture (techniciens des Chambres d'Agriculture de la Charente, de la Charente-Maritime et d'Indre et Loire et de la Vienne) sur la base d'observations réalisées par des producteurs et techniciens : Charentes- Alliance, les entreprises de production de melon, la coopérative AGROLEG, la coopérative UNIRE, des producteurs d'Agrobio Poitou-Charentes. Ce bulletin est réalisé à partir d'observations ponctuelles. Il a pour vocation de donner une tendance de la situation sanitaire régionale. Celle-ci ne peut être transposée telle quelle dans les parcelles de production légumières (conditions très variables). La Chambre Régionale d'Agriculture de Poitou-Charentes et le rédacteur dégagent toute responsabilité quant aux décisions prises par les producteurs pour la protection de leurs cultures. Elle les invite à prendre ces décisions sur la base des observation s qu'ils auront réalisées dans leurs parcelles. Les")

[('bioagressor', 0.007488250732421875), ('disease', 0.0032596588134765625)]

In [34]:
#cas disease: 0, bioagressor: 0 - french crop usasge
predictor.predict("Ensemble de plantes cultivées pour leurs fruits ou leurs graines riches en matières grasses (lipides). De ces fruits et graines sont extrait une huile à usage alimentaire humaine, alimentaire animal ou industriel. Les résidus de l'extraction constituent les tourteaux utilisés pour l'alimentation animale.")

[('bioagressor', 0.00669097900390625), ('disease', 0.0036220550537109375)]

In [35]:
#cas disease: 0, bioagressor: 0 - french crop usasge
predictor.predict("Orge semé après le 1er février, principalement de mars à mai. Orge de printemps est toujours à deux rangs")

[('bioagressor', 0.00577545166015625), ('disease', 0.004180908203125)]

In [36]:
#cas disease: 1, bioagressor: 0 - tweet - jaunisse
predictor.predict("des parcelles qui ont eu une croissance presque nulle depuis cet été en majeur partie dû à la jaunisse.")

[('disease', 0.9853515625), ('bioagressor', 0.0118255615234375)]

In [37]:
#cas disease: 1, bioagressor: 0 - tweet - sécheresse => raté, il n'a pas encore vu sécheresse, mais ok pour jaunisse ou pourriture
predictor.predict("Parfois,on nous demande. Que faites-vous,par cette sécheresse?")

[('bioagressor', 0.006145477294921875), ('disease', 0.00399017333984375)]

In [38]:
#cas disease: 0, bioagressor: 1 - tweet - pucerons

predictor.predict("Attention pucerons dans les blés")

[('bioagressor', 0.9892578125), ('disease', 0.0082855224609375)]

In [39]:
#cas disease: 1, bioagressor: 1 - Carpocapse, tavelure
predictor.predict("Favorisée par les conditions climatiques,l'activité du Carpocapse est toujours intense et il est conseillé de renouveler la protection des fruits en effectuant un nouveau traitement dès réception de ce bulletin. Tavelure du pommier: la maturité des périthèces de cette tavelure est très avancés et les prochaines pluies risquent d'être contaminatrices")

[('disease', 0.9912109375), ('bioagressor', 0.98681640625)]

In [40]:
#cas disease: 1, bioagressor: 0 tavelure
predictor.predict("tavelure du pommier: la maturité des périthèces de cette tavelure est très avancés et les prochaines pluies risquent d'être contaminatrices")

[('disease', 0.98193359375), ('bioagressor', 0.00971221923828125)]

### Analyse the fine-tuned model

In [41]:
predictor = BertClassificationPredictor(
                model_path='finetuned_model/model_out',
                label_path='labels/',
                multi_label=True,
                model_type='camembert-base',
                do_lower_case=False)

In [42]:
#evaluation on validation set
df_val = pd.read_csv('./data/val_set.csv')

#predictor.get_learner()

In [43]:
df_val.Disease = df_val.Disease.astype(int)
df_val.tail()

,Unnamed: 0,report_text,source_name,Bioagressor,Disease
75,247,doryphore -traiter les larves encore jeunes av...,AA_TC-PACA_1980_014,1,0
76,228,"~ doryphore, :au lieu de carbaryl 100 gr. de m...",AA_TC_Champagne-Ardenne_1965_003,0,0
77,371,tavelures des poiriers et des pommiers -corne ...,AA_TC_Aquitaine_1967_013,0,1
78,176,desherbage du tournesolcette culture est très ...,AA_GC_Aquitaine_1992_005,0,0
79,272,mais doux pyrale :\r\nl'activité de nymphose e...,AA_CL_Centre_1994_018,1,0


In [44]:
batch_predictions = predictor.predict_batch(df_val.report_text.to_list())

C:\Users\Raja\Documents\SJI\tools\anaconda3\envs\fast-camembert\lib\site-packages\pandas\core\frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


In [45]:
batch_predictions[:10]

[[('disease', 0.449951171875), ('bioagressor', 0.0016870498657226562)],
 [('disease', 0.005039215087890625), ('bioagressor', 0.0050201416015625)],
 [('disease', 0.99072265625), ('bioagressor', 0.98681640625)],
 [('bioagressor', 0.00757598876953125), ('disease', 0.00327301025390625)],
 [('bioagressor', 0.007122039794921875), ('disease', 0.0035648345947265625)],
 [('disease', 0.0099334716796875), ('bioagressor', 0.0032978057861328125)],
 [('bioagressor', 0.006313323974609375), ('disease', 0.0038547515869140625)],
 [('bioagressor', 0.98583984375), ('disease', 0.006412506103515625)],
 [('bioagressor', 0.007038116455078125), ('disease', 0.003749847412109375)],
 [('disease', 0.9267578125), ('bioagressor', 0.006313323974609375)]]

In [46]:
#dict(batch_predictions[1]).values()
list_y_pred = [ dict(pred) for pred in batch_predictions]
#list_y_pred[-5:]

In [47]:
df_y_pred = pd.DataFrame(list_y_pred, columns =['bioagressor', 'disease']) 
df_y_pred = df_y_pred.rename(columns={"bioagressor": "Bioagressor", "disease": "Disease"})
df_y_pred.tail()

,Bioagressor,Disease
75,0.992188,0.012337
76,0.014557,0.002108
77,0.003622,0.929688
78,0.006824,0.003565
79,0.987305,0.006824


In [48]:
df_y_pred.describe()

,Bioagressor,Disease
count,80.000000,80.000000
mean,0.346618,0.243190
std,0.466218,0.406944
min,0.001687,0.001335
25%,0.005976,0.003761
50%,0.007360,0.006220
75%,0.985840,0.375061
max,0.996582,0.993652


In [49]:
df_y_real = pd.DataFrame(df_val, columns=['Bioagressor', 'Disease'])
df_y_real.tail()

,Bioagressor,Disease
75,1,0
76,0,0
77,0,1
78,0,0
79,1,0


In [50]:
df_y_real.describe()

,Bioagressor,Disease
count,80.000000,80.000000
mean,0.237500,0.275000
std,0.428236,0.449331
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,1.000000
max,1.000000,1.000000


In [51]:
import numpy as np
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

In [52]:
precision_score(y_true=df_y_real.values, y_pred=df_y_pred.values > 0.5,average=None) #If None, the scores for each class are returned

array([0.60714286, 0.73684211])

In [53]:
precision_score(y_true=df_y_real.values, y_pred=df_y_pred.values > 0.5, average='weighted')

0.6767375756464331

In [54]:
recall_score(y_true=df_y_real.values, y_pred=df_y_pred.values > 0.5,average=None)

array([0.89473684, 0.63636364])

In [55]:
recall_score(y_true=df_y_real.values, y_pred=df_y_pred.values > 0.5,average='weighted')

0.7560975609756098

In [56]:
f1_score(y_true=df_y_real.values, y_pred=df_y_pred.values > 0.5,average=None)

array([0.72340426, 0.68292683])

In [57]:
f1_score(y_true=df_y_real.values, y_pred=df_y_pred.values > 0.5,average='weighted')

0.7016846608528358

In [58]:
accuracy_score(y_true=df_y_real.values, y_pred=df_y_pred.values > 0.5)

0.6875

In [59]:
accuracy_score(y_true=df_y_real.Bioagressor.values, y_pred=df_y_pred.Bioagressor.values > 0.5)


0.8375

In [60]:
accuracy_score(y_true=df_y_real.Disease.values, y_pred=df_y_pred.Disease.values > 0.5 )


0.8375

In [61]:
from sklearn.metrics import average_precision_score
average_precision_score(df_y_real.values, df_y_pred.values, average=None)

array([0.62402556, 0.58537994])

In [62]:
from sklearn.metrics import average_precision_score
average_precision_score(df_y_real.values, df_y_pred.values, average='weighted')

0.6032888855072924

In [63]:
df_results = pd.concat([df_val, df_y_pred > 0.5], axis=1, ignore_index=True)

In [64]:
df_results.to_csv('predictions_camembert-base_2_2.csv')